# Clase 5

Objetivos

* Alineamiento global vs alineamiento local.
* Algortimo de alineamiento local de Smith-Waterman.



## Anuncios
----------

1. En la primera clase luego del interludio haremos un repaso de todo el material para la primera evaluación.
2. En la segunda clase del mismo día haremos el control individual, el cual es un questionario.
3. Formaremos hoy los grupos para el trabajo grupal.

# Repaso de Alineamiento global


En la clase anterior aprendimos a evaluar la bondad de un alineamiento entre dos secuencias a través de la medición de su score. Si tenemos dos alineamientos, elegimos el que tenga el mayor **score**. 

Esto implica, que el dentro de todos los alineamientos posibles entre dos secuencias, el mejor es aquel con el máximo score. Esto se llama alineamiento global.

El algoritmo de **Needleman-Wuns** utiliza una matriz de substitución (por ejemplo BLOSUM50) para encontrar el alineamiento con el maximo score posible.

Antes de ver el algoritmo de **Needleman-Wunsh**, veamos maneras convenientes de representar los alineamientos.

# Representación matricial del alineamiento de dos secuencias

Las secuencias: 
```
x=AETGGW  
y=AEEH
```
Pueden ser alineadas como
```
AETGGW
A-E-EH
```
o
```
AETGGW
AEE--H
```  
o de varias otras formas.  

Para expresar todos los posibles alineamientos de manera compacta podemos hacer uso de una representación matricial. Por ejemplo:
    
```
AETGGW    AETGGW
A-E-EH    AEE--H
```
Puden ser representados de la siguiente manera:

<table><tr>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_matrixAlingment1.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_matrixAlingment1.png" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

¿Cómo podemos representar alineamientos que comienzan con un gap? Por ejemplo:

```
AETGGW    -AETGGW
-AEE-H    AE-E--H
```

Para ésto podemos agregar la siguiente modificación a nuesta matriz de alineamientos:


<table><tr>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_alignmentMatrix2.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_alignmentMatrix2.png" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

**Ejercicio 1** Representa en forma matricial los siguientes alienamientos:

1.
```
GTAAETGGW
-WTAE-E-H
```
2.
```
-ETGGTAGW
E-E--WTAH
```


**Ejercicio 2** Para verificar que entendimos como ésto funciona, ahora hagamos el proceso en sentido inverso. Dadas las siguientes matrices, representa el alineamiento como una secuencia letra a letra:

<table><tr>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_example1.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_example2.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

# Algortimo de alineamiento de Needleman-Wunsh

## ¿De qué sirve ésto? En la misma matríz podemos anotar los scores de los alineamientos parciales


Supongamos que utilizando la matriz BLOSUM50 (scores para match/mismatch) y $d=-8$ (score para gaps), queremos encontrar el mejor alineamineto entre las secuencias
```
  x=HEAGAWGHEE   y=PAWHEAE 
```


 En este caso podemos escribir todas la alineaciones posibles y los scores de los mejores alineamientos parciales como la matriz:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_exampleAlignment.jpg" alt="Drawing" style="width: 400px;" />

En donde la coordenada $(i,j)$ corresponde al mejor score entre las secuencias $x_{1...i}$ y $y_{1...j}$. 

Calculamos el valor de cada coordenadas en base a sus valores adyacentes:

<table><tr>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_exampleAlignment.jpg" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_trace.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

De estas tres opciones, se selecciona el score que hasta ese momento sea el más alto:

<table><tr>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_trace.jpg" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_selection.jpg" alt="Drawing" style="width: 450px;"/> </td>
</tr></table>


Ejercicio 4
----------

Dada las secuencias $x=HEAGAWGHEE$, $y=PAWHEAE$, escribe código para:

1. Construir la matriz de scores de cada secuencia (Matriz BLOSUM50).
2. Computar la matriz de los los mejores alineamientos parciales (Matriz F).
3. Trazar la suceción de alineamientos parciales con el mayor score total.
4. Escribe una función ```align(seq1,seq2)``` que tome como argumentos las secuencias ```seq1``` y ```seq2```, y entrgue como resultado su mejor alineamiento. 

In [18]:
import numpy as np
from Bio.SubsMat import MatrixInfo    #Importar los datos de la libreria

def getScore(a,b):
    pair = (a,b)
    if not pair in MatrixInfo.blosum50:
        pair = tuple(reversed(pair))
    return(MatrixInfo.blosum50[pair])
    
#Estas son mis dos secuencias a evaluar
x = "-HEAGAWGHEE"   # x= columnas
y = "-PAWHEAE"      # y= filas  
d = -8

T=np.zeros((len(y),len(x) ) ) # Matriz de trazabilidad
T[0,:] = len(x)*[1] # Rellena la primera fila
T[:,0] = len(y)*[2] # Rellena la primera columna

F=np.zeros((len(y),len(x) ) ) # Matriz de los maximos scores
F[0,:] = np.arange(len(x))*d # Rellena la primera fila
F[:,0] = np.arange(len(y))*d # Rellena la primera columna

for j in range(1,len(y)): # Itera entre filas
    for i in range(1,len(x)): # Itera entre columnas
        diagonal   = F[j-1,i-1] + getScore(x[i],y[j])
        lateral    = F[j  ,i-1] + d
        vertical   = F[j-1,i  ] + d
        F[j,i]     = np.max(    [diagonal,lateral,vertical] ) 
        T[j,i]     = np.argmax( [diagonal,lateral,vertical] )
print(F)
print(T)

[[  0.  -8. -16. -24. -32. -40. -48. -56. -64. -72. -80.]
 [ -8.  -2.  -9. -17. -25. -33. -41. -49. -57. -65. -73.]
 [-16. -10.  -3.  -4. -12. -20. -28. -36. -44. -52. -60.]
 [-24. -18. -11.  -6.  -7. -15.  -5. -13. -21. -29. -37.]
 [-32. -14. -18. -13.  -8.  -9. -13.  -7.  -3. -11. -19.]
 [-40. -22.  -8. -16. -16.  -9. -12. -15.  -7.   3.  -5.]
 [-48. -30. -16.  -3. -11. -11. -12. -12. -15.  -5.   2.]
 [-56. -38. -24. -11.  -6. -12. -14. -15. -12.  -9.   1.]]
[[2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0.]
 [2. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.]
 [2. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1.]
 [2. 0. 0. 0. 0. 0. 2. 0. 0. 1. 1.]
 [2. 2. 0. 1. 0. 0. 0. 2. 0. 0. 0.]
 [2. 2. 2. 0. 1. 0. 0. 0. 2. 2. 0.]
 [2. 2. 0. 2. 0. 0. 0. 0. 0. 0. 0.]]


In [21]:
# Con esto comenzamos en la esquina inferior derecha de la matriz T
i=len(x)-1
j=len(y)-1

alignedX = ""
alignedY = ""

while j>0 or i>0:
    if T[j,i]==0: #diagonal
        alignedX += x[i]
        alignedY += y[j]
        i -= 1
        j -= 1
    elif T[j,i]==1: #lateral
        alignedX += x[i] 
        alignedY += "-"
        i -= 1
    elif T[j,i]==2: #vertical
        alignedX += "-" 
        alignedY += y[j]
        j -= 1
print(alignedX[::-1])
print(alignedY[::-1])

HEAGAWGHE-E
--P-AW-HEAE
